# Search Algorithms

## Creating Graph 1: BMS, BFS, DFS (Source S, Goal F)

In [3]:
# Defining vertices and edges of the graph in the form of a list
vertices = ['S', 'A', 'B', 'C', 'D', 'E', 'F']
edges = [('S','A'),('S', 'B'), ('B', 'C'), ('B', 'D'),('D', 'F'),('C', 'E'), ('E', 'F'),('A', 'F')]

# function to convert vertex labels to numeric indices
def get_vertex_index(vertex):
    return vertices.index(vertex)

# Creating an empty adjacency matrix
num_vertices = len(vertices)
adjacency_matrix = [[0] * num_vertices for _ in range(num_vertices)]

# Fill in the adjacency matrix based on edges
for edge in edges:
    vertex1, vertex2 = edge
    i, j = get_vertex_index(vertex1), get_vertex_index(vertex2)
    adjacency_matrix[i][j] = 1  # For directed graph
    adjacency_matrix[j][i] = 1  # For undirected graph

graph = adjacency_matrix
for row in graph:
    print(row)

[0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 1]
[0, 1, 0, 0, 1, 1, 0]


## Depth First Search

In [4]:
# Defining DFS function to print the path
def dfs_with_path(graph, start, end, path=[]):
    path = path + [start]

    if start == end:
        print("Path from 'S' to 'F':", " -> ".join(vertices[vertex] for vertex in path))
    else:
        for neighbor in range(len(graph[start])):
            if graph[start][neighbor] == 1 and neighbor not in path:
                dfs_with_path(graph, neighbor, end, path)

# Finding the path taken by DFS from S to F
start_vertex = get_vertex_index('S')
end_vertex = get_vertex_index('F')
dfs_with_path(graph, start_vertex, end_vertex)


Path from 'S' to 'F': S -> A -> F
Path from 'S' to 'F': S -> B -> C -> E -> F
Path from 'S' to 'F': S -> B -> D -> F


## British Museum Search

In [6]:
from collections import deque

def british_museum_search(graph, start, goal):
    queue = deque()
    visited = set()
    path = []
    
    queue.append([start])
    
    while queue:
        current_path = queue.popleft()
        current_node = current_path[-1]
        
        if current_node == goal:
            path = current_path
            break
        
        if current_node not in visited:
            visited.add(current_node)
            for neighbor in range(len(graph[current_node])):
                if graph[current_node][neighbor] == 1:
                    new_path = list(current_path)
                    new_path.append(neighbor)
                    queue.append(new_path)
    
    return path

# Finding the path taken by British Museum Search from S to F
start_vertex = get_vertex_index('S')
end_vertex = get_vertex_index('F')
bms_path = british_museum_search(graph, start_vertex, end_vertex)

if bms_path:
    print("Path from 'S' to 'F' (British Museum Search):", " -> ".join(vertices[vertex] for vertex in bms_path))
else:
    print("No path from 'S' to 'F'")


Path from 'S' to 'F' (British Museum Search): S -> A -> F


In [7]:
from collections import deque

def bfs(graph, start, goal):
    queue = deque()
    visited = set()
    parent = {}
    
    queue.append(start)
    visited.add(start)
    
    while queue:
        current_node = queue.popleft()
        
        if current_node == goal:
            break
        
        for neighbor in range(len(graph[current_node])):
            if graph[current_node][neighbor] == 1 and neighbor not in visited:
                queue.append(neighbor)
                visited.add(neighbor)
                parent[neighbor] = current_node
    
    # Reconstruct the path from the goal to the start
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()
    
    return path

# Finding the path taken by BFS from S to F
start_vertex = get_vertex_index('S')
end_vertex = get_vertex_index('F')
bfs_path = bfs(graph, start_vertex, end_vertex)

if bfs_path:
    print("Path from 'S' to 'F' (BFS):", " -> ".join(vertices[vertex] for vertex in bfs_path))
else:
    print("No path from 'S' to 'F'")


Path from 'S' to 'F' (BFS): S -> A -> F


## Hill Climbing Search

In [8]:
# Function to perform Hill Climbing Search
def hill_climbing_search(graph, start, goal):
    current_node = start
    path = [current_node]
    
    while current_node != goal:
        neighbors = []
        for neighbor in range(len(graph[current_node])):
            if graph[current_node][neighbor] == 1:
                neighbors.append(neighbor)
        
        if not neighbors:
            break
        
        # Choosing neighbor that is closest to the goal
        closest_neighbor = min(neighbors, key=lambda neighbor: abs(neighbor - goal))
        current_node = closest_neighbor
        path.append(current_node)
    
    if path[-1] == goal:
        return path
    else:
        return None

# Finding the path taken by Hill Climbing Search from S to F
start_vertex = get_vertex_index('S')
end_vertex = get_vertex_index('F')
hill_climbing_path = hill_climbing_search(graph, start_vertex, end_vertex)

if hill_climbing_path:
    print("Path from 'S' to 'F' (Hill Climbing Search):", " -> ".join(vertices[vertex] for vertex in hill_climbing_path))
else:
    print("No path from 'S' to 'F'")


Path from 'S' to 'F' (Hill Climbing Search): S -> B -> D -> F


## Beam Search

In [10]:
from queue import PriorityQueue

# Function to perform Beam Search with uniform heuristics
def beam_search_uniform(graph, start, goal, beam_width):
    frontier = PriorityQueue()
    frontier.put((0, start))
    paths = {start: []}
    
    while not frontier.empty():
        _, current = frontier.get()
        
        if current == goal:
            return paths[current] + [current]
        
        neighbors = []
        for neighbor in range(len(graph[current])):
            if graph[current][neighbor] == 1:
                neighbors.append(neighbor)
        
        neighbors = sorted(neighbors)
        
        for neighbor in neighbors[:beam_width]:
            new_path = paths[current] + [current]
            if neighbor not in paths or len(new_path) < len(paths[neighbor]):
                frontier.put((-len(new_path), neighbor))
                paths[neighbor] = new_path
    
    return None

# Finding the path taken by Beam Search with uniform heuristics from S to F with a beam width of 2
start_vertex = get_vertex_index('S')
end_vertex = get_vertex_index('F')
beam_width = 2
beam_search_uniform_path = beam_search_uniform(graph, start_vertex, end_vertex, beam_width)

if beam_search_uniform_path:
    print("Path from 'S' to 'F' (Beam Search with Uniform Heuristics):", " -> ".join(vertices[vertex] for vertex in beam_search_uniform_path))
else:
    print("No path from 'S' to 'F'")


Path from 'S' to 'F' (Beam Search with Uniform Heuristics): S -> A -> F
